In [2]:
import pandas as pd
from IPython.display import HTML, clear_output


df = pd.read_json('regen.json')
df.fillna(value={'curation_status': 0, 'curation_message': ''}, inplace=True)
df.tail(3)

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
197,Create a vibrant website for a music festival ...,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",{'Create a visually appealing website for a fi...,0.316872,1,
198,Create a clean and modern website for a photog...,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",{'Create an elegant and minimalist photography...,0.355689,1,
199,Create a modern and minimalist website for a p...,"<!DOCTYPE html>\n<html lang=""en"">\n <head>\n ...",{'Create a website for a photography portfolio...,0.417742,1,


In [155]:
curation_df = pd.DataFrame(columns=['curation_status', 'curation_message'])

In [156]:
for idx, row in df[df['curation_status'] == 0].iterrows():
  last_idx = idx
  instruction, output, most_similar_instructions, avg_similarity_score, _, __ = row
  stop = False
  while not stop:
    clear_output(wait=True)
    display(HTML(output))
    
    print(f'''Instruction: {instruction}
    Output: {output}''')
    
    curator_input = input(f'{idx}. Is the data good? (yes/no/mark/stop):')
    curator_input = curator_input.strip()
    if curator_input == 'yes':
      curation_df.loc[idx] = {
        'curation_status': 1,
        'curation_message': '',
      }
      break
    elif curator_input == 'no':
      curation_df.loc[idx] = {
        'curation_status': -1,
        'curation_message': '',
      }
      break
    # Mark the data if you think the data can be fixed/improved manually
    elif curator_input == 'mark':
      suggestion = input('Suggestion: ')
      curation_df.loc[idx] = {
        'curation_status': 0,
        'curation_message': suggestion,
      }
      break
    elif curator_input == 'stop':
      stop = True
    else:
      print('Invalid input:', '"' + curator_input + '"')
  if stop:
    break
clear_output()

In [157]:
curation_df

,curation_status,curation_message
199,-1,
200,1,


In [158]:
curated_df = curation_df.combine_first(df)[df.columns]
curated_df.iloc[curation_df.index]

,instruction,output,most_similar_instructions,avg_similarity_score,curation_status,curation_message
199,Create a stylish website for an online clothin...,"<!DOCTYPE html>\n<html lang=""en"">\n\n<head>\n ...",{'Build a website for an online store that sel...,0.413063,-1,
200,Create a modern and minimalist website for a p...,"<!DOCTYPE html>\n<html lang=""en"">\n\n<head>\n ...",{'Create a website for a photography portfolio...,0.417742,1,


In [159]:
curated_df[curated_df['curation_status'] != -1].to_json('regen-curated.json', orient="records")
print('regen-curated.json exported sucessfully!')

regen-curated.json exported sucessfully!


In [3]:
ok_df = pd.read_json('regen.json')
ok_df[['instruction', 'output']].to_json('webgen-dataset-2.json', orient='records')
print('webgen-dataset-2.json exported sucessfully!')

webgen-dataset-2.json exported sucessfully!
